# Calculating the penalty

In [1]:
import numpy as np 
import librosa
import time
import os
# reduce number of threads to a typical quad core computer (8 threads)
# I am running this on a computer with a large number of threads, 
# so it would be unfair to compare our serial preprocessing to tensorflow, 
# which will very efficiently use a very high number of cores.
os.environ['TF_NUM_INTEROP_THREADS'] = '8'
os.environ['TF_NUM_INTRAOP_THREADS'] = '8'
from pathlib import Path 
datadir = Path('datadir')
datafile = datadir / 'audVisIdn.npz'

# Benchmarking our CPU

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
#!wget https://upload.wikimedia.org/wikipedia/commons/f/f9/Zoorashia_elephant.jpg -O elephant.jpg
numTry = 100
numParam = 25.6e6

with tf.device('/cpu:0'): # if using gpu for preprocessing /gpu:0
  model = ResNet50(weights='imagenet')  # num of params: 25.6 M
  img_path = 'elephant.jpg'
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  preds = model.predict(x)

  tic = time.perf_counter()
  for i in range(numTry):
    preds = model.predict(x, verbose=0)

  toc = time.perf_counter()
  totTime = toc - tic
  perRunTime = totTime / numTry
  paramSpeed = numParam / perRunTime 
  # time your preprocessing operation for one sample and multiply with this
  # value to calculate parameter count equivalency 
  print("number of parameters per second:", paramSpeed)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 1s 535ms/step
number of parameters per second: 328755710.1447682


In [3]:
from threading import Thread

def getMFCCs(dic):
  X = dic['X']
  rate = dic['rate']
  mfcctot = librosa.feature.mfcc(y=X, sr=rate, n_mfcc=60)
  mfccs = np.median(mfcctot.T,axis=0).T
  dic['mfccs'] = mfccs

def getSTFT(dic):
  X = dic['X']
  dic['stft'] = np.abs(librosa.stft(X))

def getMel(dic):
  X = dic['X']
  rate = dic['rate']
  dic['mel'] = np.median(librosa.feature.melspectrogram(X, sr=rate).T,axis=0).T

def getChroma(dic):
  stft = dic['stft']
  rate = dic['rate']
  dic['chroma'] = np.median(librosa.feature.chroma_stft(S=stft, sr=rate).T,axis=0).T

def getContrast(dic):
  stft = dic['stft']
  rate = dic['rate']
  dic['contrast'] = np.median(librosa.feature.spectral_contrast(S=stft, sr=rate).T,axis=0).T


def audioFeatures(X, rate=16000):
    dic = {}
    dic['X'] = X
    dic['rate'] = rate
    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    p1 = Thread(target=getMFCCs, args=(dic,))
    p1.start()

    
    #mel-scaled spectrogram:
    p2 = Thread(target=getMel, args=(dic,))
    p2.start()
    
    # the stft, base of the last two features
    p3 = Thread(target=getSTFT, args=(dic,))
    p3.start()
    # wait for stft to complete before 
    # calculating the last 2.
    p3.join()

    # chroma:
    p4 = Thread(target=getChroma, args=(dic,))
    p4.start()

    # contrast:
    p5 = Thread(target=getContrast, args=(dic,))
    p5.start()
    
    # wait for everyone
    p1.join()
    p2.join()
    p4.join()
    p5.join()
    
    # here are the features.
    return np.concatenate((dic['mfccs'], 
                           dic['chroma'], 
                           dic['mel'], 
                           dic['contrast']), axis=1)

class AudioData():
    def __init__(self, path):
        self.file = path
        self.prepareData()

    def prepareData(self):
        ffile = np.load(self.file)
        self.tst_audio = ffile['audioTrs_test']
           

a = AudioData(datafile)






t0 = time.time()
_ = audioFeatures(a.tst_audio[:numTry])

tpersample =  ( time.time() - t0) / numTry


print(f"equivalent # of params: {paramSpeed * tpersample:.0f}")
print(f"that is a penalty of {paramSpeed * tpersample/1e7:.02f}%")

<ipython-input-3-7d34a0744b23>:17: FutureWarning: Pass y=[[-0.03475952 -0.0295105  -0.04678345 ... -0.02377319 -0.02432251
  -0.02520752]
 [ 0.04412842  0.05596924  0.05584717 ...  0.00021362  0.00112915
   0.00140381]
 [-0.02783203 -0.02276611 -0.02420044 ...  0.1546936   0.17932129
   0.15518188]
 ...
 [-0.02749634 -0.01629639 -0.00094604 ...  0.00125122  0.00921631
   0.00222778]
 [ 0.00509644  0.00726318  0.0083313  ... -0.00268555 -0.00268555
  -0.00271606]
 [-0.01239014 -0.03939819 -0.0640564  ... -0.13027954 -0.1138916
  -0.10131836]] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  dic['mel'] = np.median(librosa.feature.melspectrogram(X, sr=rate).T,axis=0).T


equivalent # of params: 1843863
that is a penalty of 0.18%


# Now image preprocessing

In [4]:
from skimage.transform import resize
import dlib


class ImageData():
    def __init__(self, path):
        self.file = path
        self.prepareData()

    def prepareData(self):
        ffile = np.load(self.file)
        self.testimg = ffile['imgFrames_test'] / 255.

# Load the detector
detector = dlib.get_frontal_face_detector()
FINALSIZE = 160

g = ImageData(datafile)

def extractFace(index):
    img = g.testimg[index,:,:,:]
        
    faces = detector((255*img).astype(np.uint8))
    
    for face in faces[:1]:
        x1 = face.left() # left point
        y1 = face.top() # top point
        x2 = face.right() # right point
        y2 = face.bottom() # bottom point
        
        lenx = (x2-x1)
        leny = (y2-y1)
        diff = lenx - leny
        # make the crop a square:
        if diff > 0:
            x1 += diff // 2 
            x2 -= diff // 2
        if diff < 0 :
            y1 += diff // 2
            y2 -= diff // 2
    if len(faces) == 0:
        # well....then no crop.
        imgr = img
    else:
        # found a face? draw a square.
        imgr = img[y1:y2, x1:x2]
        
    try:
        imgr = resize(imgr, (FINALSIZE, FINALSIZE))
    except:
        imgr = resize(img, (FINALSIZE, FINALSIZE))
        
    mean, std = np.nanmean(imgr), np.nanstd(imgr)
    imgr =  (imgr - mean) / std

    return imgr



t0 = time.time()
for i in range(numTry):
    _ = extractFace(i)
t = time.time() - t0
t /= numTry
print(f"equivalent # of params: {paramSpeed * t:.0f}")
print(f"that is a penalty of {paramSpeed * t/1e7:.02f}%")

equivalent # of params: 2001385
that is a penalty of 0.20%
